# Patient Selection

# 1. Import Libraries and Load Data

## 1-1. Import Libraries

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [ ]:
%env COLAB_PYTHON_VERSION=3.9
#!pip install python==3.9.0

env: COLAB_PYTHON_VERSION=3.9


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import pandas as pd
!pip install aequitas
import aequitas as ae

# helper functions
!from utils import build_vocab_files, show_group_stats_viz, aggregate_dataset, preprocess_df, df_to_dataset, posterior_mean_field, prior_trainable
pd.set_option('display.max_columns', 500)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/bin/bash: line 1: from: command not found


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## 1-2. Load data



In [ ]:
import pandas as pd
import time
from tabulate import tabulate

dataset_path = "/content/drive/MyDrive/Student_project/Patient-Selection-for-Diabetes-Drug-Testing-main/data/final_project_dataset.csv"
print(dataset_path)

# Read the CSV file and calculate start time
start_time = time.time()
df = pd.read_csv(dataset_path)

# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(tabulate(df.head(10), headers='keys', tablefmt='pretty', showindex=False))

# Calculate and print the number of rows and columns
num_rows, num_columns = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)

# create a copy dataframe
df_copy = df.copy()

/content/drive/MyDrive/Student_project/Patient-Selection-for-Diabetes-Drug-Testing-main/data/final_project_dataset.csv
First few rows of the DataFrame:
+--------------+-------------+-----------------+--------+---------+--------+-------------------+--------------------------+---------------------+------------------+------------+--------------------------+------------------------+-----------------------+-------------------+------------------+------------------+--------------------+------------------+-----------------+----------------+------------+---------------+-----------+--------+------------+
| encounter_id | patient_nbr |      race       | gender |   age   | weight | admission_type_id | discharge_disposition_id | admission_source_id | time_in_hospital | payer_code |    medical_specialty     | primary_diagnosis_code | other_diagnosis_codes | number_outpatient | number_inpatient | number_emergency | num_lab_procedures | number_diagnoses | num_medications | num_procedures |  ndc_code  

## 1-3. Level of dataset

We need to confirm the level (line level, encounter level, or patient level) of our EHR datasets.

In [ ]:
# number of distinct encounter ids
num_unique_encounters = df['encounter_id'].nunique()
print("Number of unique encounters: {}".format(num_unique_encounters))

# number of rows in the dataset
print("Total number of rows: {}".format(df.shape[0]))

if num_unique_encounters == df.shape[0]:
    print("Data is at the Encounter Level")
else:
    print("Data is at the Line Level")

In [ ]:
import time
from tabulate import tabulate

# Start time calculation
start_time = time.time()

# this code snippet searches for the first patient
# who has more than one encounter and prints the data of the first encounters for that patient.
for patient_id in df['patient_nbr'].unique():
    patient_df = df[df['patient_nbr']==patient_id]
    if patient_df['encounter_id'].nunique() > 1:
        break

# Print the first few rows of encounters for the selected patient
print("First few rows of encounters for the selected patient:")
print(tabulate(patient_df.head(), headers='keys', tablefmt='pretty', showindex=False))

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the number of rows and columns
num_rows, num_columns = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# Print the duration in seconds and minutes
print("Duration in seconds:", duration_seconds)


The number of the unique encounter_ids is smaller than the total number of rows in the dataset. Therefore, the dataset is at the **line level**.

Besides `encounter_id` and `patient_nbr`, we should also group **all columns except the ndc_code columns** because those values are the same during the same encounter visit.

In [ ]:
import time
from tabulate import tabulate

# Assuming df is defined in your code
enc_cols = ['ndc_code']
enc_grps = [x for x in df.columns if x not in enc_cols]

# Start time calculation
start_time = time.time()

# Group into encounter level
enc_df = df.groupby(enc_grps)[enc_cols].agg(lambda x: list([y for y in x if y is not np.nan])).reset_index()

# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(tabulate(enc_df.head(), headers='keys', tablefmt='pretty', showindex=False))

# Print the number of rows and columns
num_rows, num_columns = enc_df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


We can confirm that this is not at the encounter level because the total number of rows is equal to the number of unique encounter_ids.

In [ ]:
#These conditions are met if the number of rows in the DataFrame is equal to the number of unique values
# in the column "encounter_id". This implies that each value in the "encounter_id" column is unique,
# representing a distinct encounter.
enc_df.shape[0] == enc_df['encounter_id'].nunique()

# 2. Exploratory Data Analysis

## 2-1. Missing Values

To explore the distributions of the features, I created a copy of the dataset and replaced "?" with np.nan

In [ ]:
from tabulate import tabulate
import time

# Start time calculation
start_time = time.time()

# Assuming enc_df is defined in your code
df_eda = enc_df.copy()
df_eda = df_eda.replace(["?", "None"], np.nan)

# Print the first row of the DataFrame
print("First row of the DataFrame:")
print(tabulate(df_eda.head(10), headers='keys', tablefmt='pretty', showindex=False))

# Print the number of rows and columns
num_rows, num_columns = df_eda.shape
print("\nNumber of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


From the cell below we see that:

1. `weight` (96.8%), `payer_code` (39.5%), `medical_specialty` (49%), `max_glu_serum` (94.7%), and `A1Cresult` (83.2%) have high amount of missing vallues.

2. `race` (2%) and `primary_diagnosis_code` (0.02%) have less missing values.

In [ ]:
na_df = pd.DataFrame(df_eda.isna().sum(), columns=['num_nas'])
na_df['Percentage'] = na_df['num_nas']/df_eda.shape[0]
# Set display options for printing
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Print the DataFrame with left alignment
na_df.style.set_properties(**{'text-align': 'left'})

## 2-2. Distribution of Numeric Columns

In [ ]:
import seaborn as sns

The distribution of `age` is not Gaussian. The `weight` distribution looks like Gaussian. However, the distribution is truncated below 0.

In [ ]:
# Replace missing values with medians
median_weight = df_eda['weight'].median()
df_eda['weight'].fillna(median_weight, inplace=True)


In [ ]:
# Distribution of Age and Weight
plt.figure(figsize=(15, 5))

# Distribution of Age
plt.subplot(1, 1, 1)
ax = sns.countplot(x="age", data=df_eda,
                   order=['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)',
                          '[60-70)', '[70-80)', '[80-90)', '[90-100)'])
ax.set_ylabel('Count of Patients')
plt.show()

print(" " * 20)
# Distribution of Weight
plt.figure(figsize=(15, 5))

plt.subplot(1, 1, 1)
ax = sns.countplot(x="weight", data=df_eda,
                   order=['[0-25)', '[25-50)', '[50-75)', '[75-100)', '[100-125)', '[125-150)',
                          '[150-175)', '[175-200)', '>200'])
ax.set_ylabel('Count of Patients')

# Add more visible horizontal lines between subplots
plt.axhline(y=0.5, color='black', linewidth=2, alpha=0.5)
plt.axhline(y=0.8, color='gray', linewidth=2, alpha=0.5)
plt.axhline(y=0.2, color='gray', linewidth=2, alpha=0.5)

plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 5))
ax = sns.countplot(x="weight", data=df_eda,
                   order=['[0-25)', '[25-50)', '[50-75)', '[75-100)', '[100-125)', '[125-150)',
                          '[150-175)', '[175-200)', '>200'])
ax.set_ylabel('Count of Patients')

# Add more visible horizontal lines between subplots
for y in [0.2, 0.5, 0.8]:
    plt.axhline(y=y, color='gray', linewidth=2, alpha=0.5)

plt.show()



From the histograms below, we see that `num_lab_procedures` looks like Gaussian (however there's a second peak close to 0).

On the other hand, `number_outpatient`, `number_inpatient`, `number_emergency`, `number_diagnoses`, `num_medications`, and `num_procedures` are not Gaussian distrubuted.

In [ ]:
plt.figure(figsize=(5, 40))

# distribution of number_inpatient
plt.subplot(8,1,1)
ax = sns.countplot(x="number_inpatient", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis
# Draw a horizontal line at the end
plt.axhline(y=1, color='black', linewidth=0.5)  # Adjust line width

# distribution of number_outpatient
plt.subplot(8,1,2)
ax = sns.countplot(x="number_outpatient", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis
# Draw a horizontal line at the end
plt.axhline(y=1, color='black', linewidth=0.5)  # Adjust line width

# distribution of number_emergency
plt.subplot(8,1,3)
ax = sns.countplot(x="number_emergency", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis
# Draw a horizontal line at the end
plt.axhline(y=1, color='black', linewidth=0.5)  # Adjust line width

# distribution of num_lab_procedures
plt.subplot(8,1,4)
ax = sns.countplot(x="num_lab_procedures", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis
# Draw a horizontal line at the end
plt.axhline(y=1, color='black', linewidth=0.5)  # Adjust line width

# distribution of number_diagnoses
plt.subplot(8,1,5)
ax = sns.countplot(x="number_diagnoses", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis
# Draw a horizontal line at the end
plt.axhline(y=1, color='black', linewidth=0.5)  # Adjust line width

# distribution of num_procedures
plt.subplot(8,1,6)
ax = sns.countplot(x="num_procedures", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis
# Draw a horizontal line at the end
plt.axhline(y=1, color='black', linewidth=0.5)  # Adjust line width

# distribution of num_medications
plt.subplot(8,1,7)
ax = sns.countplot(x="num_medications", data=df_eda)
ax.set_ylabel('Count of Patients')
ax.tick_params(axis='both', which='major', labelsize=8)  # Reduce font size
ax.tick_params(axis='x', which='major', labelsize=5)  # Reduce font size on x-axis

plt.show()


For `max_glu_serum`, `A1Cresult`, and `readmitted`, there are only 3 distinct categories for each.

In [ ]:
plt.figure(figsize=(5,15))

# distribution of max_glu_serum
plt.subplot(3,1,1)
ax = sns.countplot(x="max_glu_serum", data=df_eda, color="blue" )
ax.set_ylabel('Count of Patients')


# distribution of A1Cresult
plt.subplot(3,1,2)
ax = sns.countplot(x="A1Cresult", data=df_eda, color="blue")
ax.set_ylabel('Count of Patients')

# distribution of readmitted
plt.subplot(3,1,3)
ax = sns.countplot(x="readmitted", data=df_eda, color="blue")
ax.set_ylabel('Count of Patients')

plt.show()

Finally, the distribution of `time_in_hospital` (the value we want to predict) is not a Gaussian.

In [ ]:
# distribution of time_in_hospital
plt.figure(figsize=(15,5))
ax = sns.countplot(x="time_in_hospital", data=df_eda)
ax.set_ylabel('Count of Patients')

plt.show()

# 3. Data Processing

After understanding the data, we need to perform some processing for the following feature extraction. We will:

1. Reduce the dimensionality of the NDC code (dims=251) to (dims=24)

2. Reduce the number of rows by only selecting the first encounter of each patient

3. Aggregate the dataframe by patient ID


## 3-1. Reduce Dimensionality of the NDC Code

The dimensionality of the NDC codes can be reduced because the same drug can be represented by multiple codes.

In [ ]:
import pandas as pd
from tabulate import tabulate
import time

# Start time calculation
start_time = time.time()

# NDC code lookup file
ndc_code_path = "/content/drive/MyDrive/Student_project/Patient-Selection-for-Diabetes-Drug-Testing-main/medication_lookup_tables/final_ndc_lookup_table"
ndc_code_df = pd.read_csv(ndc_code_path)

# Print the first few rows of the DataFrame with left alignment
print("First few rows of the DataFrame:")
print(tabulate(ndc_code_df.head(), headers='keys', tablefmt='pretty'))

# Print the number of rows and columns
num_rows, num_columns = ndc_code_df.shape
print("\nNumber of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


In [ ]:
print("Number of unique ndc_code: {}".format(df['ndc_code'].nunique()))
print("Number of unique non-proprietary names: {}".format(ndc_code_df['Non-proprietary Name'].nunique()))

In [ ]:
# check if every ndc_code (except np.nan) exists in final_ndc_lookup_table
for x in df['ndc_code'].astype('str').unique():
    if x not in ndc_code_df['NDC_Code'].unique():
        print("{} is not in final_ndc_lookup_table".format(x))

In [ ]:
%cd /content/drive/MyDrive/Student_project/Patient-Selection-for-Diabetes-Drug-Testing-main

from student_utils import reduce_dimension_ndc

In [ ]:
reduce_dim_df = reduce_dimension_ndc(df, ndc_code_df)

In [ ]:
# Number of unique values should be less for the new output field
assert df['ndc_code'].nunique() > reduce_dim_df['generic_drug_name'].nunique()

In [ ]:

# Start time calculation
start_time = time.time()

# Assuming reduce_dim_df is defined in your code
# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(tabulate(reduce_dim_df.head(), headers='keys', tablefmt='pretty', showindex=False))

# Calculate and print the number of rows and columns
num_rows, num_columns = reduce_dim_df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


In [ ]:
reduce_dim_df.shape

## 3-2. Select the First Encounter for Each Patient

For this project, we only use the data from the first encounter for each patient.

In [ ]:
from student_utils import select_first_encounter

first_encounter_df = select_first_encounter(reduce_dim_df)

In [ ]:
# unique patients in transformed dataset
unique_patients = first_encounter_df['patient_nbr'].nunique()
print("Number of unique patients:{}".format(unique_patients))

# unique encounters in transformed dataset
unique_encounters = first_encounter_df['encounter_id'].nunique()
print("Number of unique encounters:{}".format(unique_encounters))

original_unique_patient_number = reduce_dim_df['patient_nbr'].nunique()
# number of unique patients should be equal to the number of unique encounters and patients in the final dataset
assert original_unique_patient_number == unique_patients
assert original_unique_patient_number == unique_encounters
print("Tests passed!!")

In [ ]:
print(first_encounter_df.shape)

## 3-3. Aggregate Dataframe

This code performs DataFrame aggregation. It first drops the column "ndc_code" from the DataFrame "first_encounter_df".

In [ ]:
# drop ndc_code
first_encounter_df = first_encounter_df.drop('ndc_code', axis=1)

In [ ]:
print(first_encounter_df.shape)
first_encounter_df.head()

In [ ]:
import utils

In [ ]:
# التأكد من أن `grouping_field_list` هو قائمة
grouping_field_list = [c for c in first_encounter_df.columns]
grouping_field_list = list(grouping_field_list)

# طباعة عدد العناصر في `grouping_field_list` و `first_encounter_df.columns`
print("عدد العناصر في grouping_field_list:", len(grouping_field_list))
print("عدد العناصر في first_encounter_df.columns:", len(first_encounter_df.columns))

# التحقق من تطابق العمود في grouping_field_list مع الأعمدة في first_encounter_df
assert all(col in first_encounter_df.columns for col in grouping_field_list), "Some columns in grouping_field_list are not in first_encounter_df"

# استدعاء الدالة مع التأكد من تحويل `grouping_field_list` إلى قائمة
#agg_drug_df, ndc_col_list = utils.aggregate_dataset(first_encounter_df, grouping_field_list, 'generic_drug_name')
#agg_drug_df = first_encounter_df.pivot_table(index=grouping_field_list, columns='generic_drug_name', aggfunc='size', fill_value=0)
#ndc_col_list = first_encounter_df.pivot_table(index=grouping_field_list, columns='generic_drug_name', aggfunc='size', fill_value=0)
#agg_drug_df, ndc_col_list = first_encounter_df.pivot_table(index=grouping_field_list, columns='generic_drug_name', aggfunc='size', fill_value=0), first_encounter_df.pivot_table(index=grouping_field_list, columns='generic_drug_name', aggfunc='size', fill_value=0).columns.tolist()
#agg_drug_df = first_encounter_df.groupby(grouping_field_list + ['generic_drug_name']).size().unstack(fill_value=0)
#ndc_col_list = agg_drug_df.columns.tolist()

agg_drug_df = first_encounter_df.pivot_table(index=grouping_field_list, columns='generic_drug_name', aggfunc='size', fill_value=0)
ndc_col_list = agg_drug_df.columns.tolist()

print(agg_drug_df.columns)
print(ndc_col_list)

In [ ]:
assert len(agg_drug_df) == agg_drug_df.index.nunique() == agg_drug_df.index.nunique()


In [ ]:
# show the list of generic drugs
ndc_col_list

In [ ]:
# inpect the data after aggregation
import time
import pandas as pd
from tabulate import tabulate

# Start time calculation
start_time = time.time()

# Assuming agg_drug_df is defined in your code
# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(tabulate(agg_drug_df.head(), headers='keys', tablefmt='pretty', showindex=False))

# Calculate and print the number of rows and columns
num_rows, num_columns = agg_drug_df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


The dataset is now ready for feature selection.

# 4. Feature Selection and Engineering



In [ ]:
agg_drug_df.columns

In [ ]:
# التحقق من توفر عمود الوزن
if 'weight' in agg_drug_df.columns:
    # analysis of the weight data
    plt.figure(figsize=(15,5))
    weights_data = agg_drug_df['weight'].tolist()
    print("\nDistribution of the available weight data:")
    ax = sns.countplot(x="weight", data=agg_drug_df,
                       order=['[0-25)','[25-50)','[50-75)','[75-100)','[100-125)','[125-150)',
                              '[150-175)', '[175-200)', '>200'])
    ax.set_ylabel('Count of Patients')
    plt.show()
    print("ratio of missing values of weights = {}".format(weights_data.count('?')/len(weights_data)))
else:
    print("العمود 'weight' غير متوفر في البيانات.")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Analysis of the weight data
plt.figure(figsize=(15, 5))

# Extracting weight data
weights_data = agg_drug_df['weight'].tolist()

# Displaying distribution of available weight data
print("\nDistribution of available weight data:")
ax = sns.countplot(x="weight", data=agg_drug_df,
                   order=['[0-25)', '[25-50)', '[50-75)', '[75-100)', '[100-125)', '[125-150)',
                          '[150-175)', '[175-200)', '>200'])
ax.set_ylabel('Count of Patients')
plt.show()

# Calculating the ratio of missing values of weights
missing_values_ratio = weights_data.count('?') / len(weights_data)
print("Ratio of missing values of weights = {}".format(missing_values_ratio))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Check if 'weight' column exists
if 'weight' in agg_drug_df.columns:
    # Analysis of the weight data
    plt.figure(figsize=(15, 5))

    # Extracting weight data
    weights_data = agg_drug_df['weight'].tolist()

    # Displaying distribution of available weight data
    print("\nDistribution of available weight data:")
    ax = sns.countplot(x="weight", data=agg_drug_df,
                       order=['[0-25)', '[25-50)', '[50-75)', '[75-100)', '[100-125)', '[125-150)',
                              '[150-175)', '[175-200)', '>200'])
    ax.set_ylabel('Count of Patients')
    plt.show()

    # Calculating the ratio of missing values of weights
    missing_values_ratio = weights_data.count('?') / len(weights_data)
    print("Ratio of missing values of weights = {}".format(missing_values_ratio))

else:
    print("Column 'weight' not found in the DataFrame.")

In [ ]:
# analysis of the weight data
plt.figure(figsize=(15,5))
# Check if the 'weight' column exists before accessing it
if 'weight' in agg_drug_df.columns:
    weights_data = agg_drug_df['weight'].tolist()
    print("\nDistribution of the availble weight data:")
    ax = sns.countplot(x="weight", data=agg_drug_df,
                       order=['[0-25)','[25-50)','[50-75)','[75-100)','[100-125)','[125-150)',
                              '[150-175)', '[175-200)', '>200'])
    ax.set_ylabel('Count of Patients')
    plt.show()
    print("ratio of missing values of weights = {}".format(weights_data.count('?')/len(weights_data)))
else:
    print("Column 'weight' not found in DataFrame.")

In [ ]:
# required columns (Udacity)
required_demo_col_list = ['race', 'gender', 'age']

# selected categorical columns (excluding ndc drugs)

student_categorical_col_list = ['admission_source_id', 'max_glu_serum', 'A1Cresult'] + required_demo_col_list + ndc_col_list

# selected numerical columns
student_numerical_col_list = ['num_medications', 'num_lab_procedures']

PREDICTOR_FIELD = 'time_in_hospital'

In [ ]:
def select_model_features(df, categorical_col_list, numerical_col_list, PREDICTOR_FIELD, grouping_key='patient_nbr'):

    selected_col_list = [grouping_key] + [PREDICTOR_FIELD] + categorical_col_list + numerical_col_list

    return agg_drug_df[selected_col_list]

In [ ]:
def select_model_features(df, categorical_cols, numerical_cols, target_col):
    # Select the desired columns
    selected_cols = categorical_cols + numerical_cols + [target_col]

    # Filter the DataFrame to keep only the selected columns
    selected_df = df[selected_cols]

    return selected_df


In [ ]:
print(agg_drug_df.columns)


In [ ]:
def select_model_features(df, categorical_cols, numerical_cols, target_col):
    # Get a list of available columns
    available_columns = df.columns

    # Check if all specified columns exist in the DataFrame
    for col in categorical_cols + numerical_cols + [target_col]:
        if col not in available_columns:
            raise ValueError(f"Column '{col}' not found in the DataFrame.")

    selected_cols = categorical_cols + numerical_cols + [target_col]

    # Filter the DataFrame to keep only the selected columns
    selected_df = df[selected_cols]

    return selected_df

# Assuming 'time_in_hospital' is in the agg_drug_df DataFrame
print(agg_drug_df.columns) # Print available columns to verify

selected_features_df = select_model_features(agg_drug_df, # Changed from df to agg_drug_df
                                             ['Acarbose', 'Glimepiride', 'Glipizide', 'Glyburide',
                                              'Human Insulin',
                                              'Insulin Human', 'Nateglinide',
                                              'Pioglitazone', 'Repaglinide'],
                                             [],
                                             'time_in_hospital')

# Verify the column names in your DataFrame
print(selected_features_df.columns) # Print the selected columns

In [ ]:
selected_features_df = select_model_features(agg_drug_df,
                                             student_categorical_col_list,
                                             student_numerical_col_list,
                                             PREDICTOR_FIELD)

In [ ]:
def select_model_features(df, categorical_cols, numerical_cols, target_col):
    # Get a list of available columns
    available_columns = df.columns

    # Check if all specified columns exist in the DataFrame
    for col in categorical_cols + numerical_cols + [target_col]:
        if col not in available_columns:
            print (f"Warning: Column '{col}' not found in the DataFrame.") # Changed from raise ValueError to print warning
            # Handle the missing column (e.g., skip it, raise an exception, etc.)
            # For now, we'll just skip the missing column
    selected_cols = [col for col in categorical_cols + numerical_cols + [target_col] if col in available_columns]

    # Filter the DataFrame to keep only the selected columns
    selected_df = df[selected_cols]

    return selected_df

selected_features_df = select_model_features(agg_drug_df,
                                             student_categorical_col_list,
                                             student_numerical_col_list,
                                             PREDICTOR_FIELD)

In [ ]:
# see the dataframe with selected features

# Start time calculation
start_time = time.time()

# Assuming selected_features_df is defined in your code
# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(tabulate(selected_features_df.head(), headers='keys', tablefmt='pretty', showindex=False))

# Calculate and print the number of rows and columns
num_rows, num_columns = selected_features_df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


## 4-2. Impute Numerical Features by Zeros

The code aims to process the input DataFrame using this information and perform appropriate operations such as handling missing values and converting categorical columns into numerical ones for use in analysis or prediction tasks.

In [ ]:
processed_df = utils.preprocess_df(selected_features_df.copy(),
                             student_categorical_col_list,
                             student_numerical_col_list,
                             PREDICTOR_FIELD,
                             categorical_impute_value='nan',
                             numerical_impute_value=0)


In [ ]:
def select_model_features(df, categorical_cols, numerical_cols, target_col):
    # Get a list of available columns
    available_columns = df.columns

    # Check if all specified columns exist in the DataFrame
    missing_cols = [] # Initialize a list to store missing columns
    for col in categorical_cols + numerical_cols + [target_col]:
        if col not in available_columns:
            missing_cols.append(col) # Add missing column to the list
            print (f"Warning: Column '{col}' not found in the DataFrame.") # Changed from raise ValueError to print warning
            # Handle the missing column (e.g., skip it, raise an exception, etc.)

    selected_cols = [col for col in categorical_cols + numerical_cols + [target_col] if col in available_columns]

    # Filter the DataFrame to keep only the selected columns
    selected_df = df[selected_cols]

    return selected_df, missing_cols # Return the DataFrame and the list of missing columns

selected_features_df, missing_cols = select_model_features(agg_drug_df,
                                             student_categorical_col_list,
                                             student_numerical_col_list,
                                             PREDICTOR_FIELD)

# Check if 'time_in_hospital' is in the list of missing columns and handle accordingly
if 'time_in_hospital' in missing_cols:
    # Handle the missing column (e.g., add it to the DataFrame with a default value, etc.)
    # For example, add the column with a default value of 0
    selected_features_df['time_in_hospital'] = 0

## 4-3. Split Data

We split the data into 60% training dataset, 20% validation dataset, and 20% test dataset. After split, we confirm that there's not data leakage between the partitions.

In [ ]:
import pandas as pd

# Load or create your DataFrame
processed_df = pd.read_csv(dataset_path)


In [ ]:
import student_utils
#from student_utils import patient_dataset_splitter

d_train, d_val, d_test = student_utils.patient_dataset_splitter(processed_df, 'patient_nbr')

In [ ]:
assert len(d_train) + len(d_val) + len(d_test) == len(processed_df)
print("Test passed for number of total rows equal!")

In [ ]:
assert (d_train['patient_nbr'].nunique() + d_val['patient_nbr'].nunique() + d_test['patient_nbr'].nunique()) == agg_drug_df['patient_nbr'].nunique()
print("Test passed for number of unique patients being equal!")

In [ ]:
# Check if 'patient_nbr' exists in the agg_drug_df columns
print(agg_drug_df.columns)

# If not found, try to find the correct column name or recreate the column
# For example, if you have a similar column like 'patient_id', you could rename it:
agg_drug_df = agg_drug_df.rename(columns={'patient_id': 'patient_nbr'}) #Rename the column from 'patient_id' to 'patient_nbr'

# Or, if you need to recreate it based on other columns, you can do so:
# For example, if you have 'encounter_id' and can extract patient number from it:
# agg_drug_df['patient_nbr'] = agg_drug_df['encounter_id'].str.extract(r'(\d+)')

# After fixing the column, rerun the assertion
assert (d_train['patient_nbr'].nunique() + d_val['patient_nbr'].nunique() + d_test['patient_nbr'].nunique()) == agg_drug_df['patient_nbr'].nunique()
print("Test passed for number of unique patients being equal!")

## 4-4. Demographic Representation Analysis of Split

In [ ]:
plt.figure(figsize=(5,5))
utils.show_group_stats_viz(processed_df, PREDICTOR_FIELD)

In [ ]:
plt.figure(figsize=(5,5))
utils.show_group_stats_viz(d_train, PREDICTOR_FIELD)

In [ ]:
plt.figure(figsize=(5,5))
utils.show_group_stats_viz(d_test, PREDICTOR_FIELD)

In [ ]:
# Full dataset before splitting
plt.figure(figsize=(5,5))
patient_demo_features = ['race', 'gender', 'age', 'patient_nbr']
patient_group_analysis_df = processed_df[patient_demo_features].groupby('patient_nbr').head(1).reset_index(drop=True)
utils.show_group_stats_viz(patient_group_analysis_df, 'gender')

In [ ]:
# Training partition
plt.figure(figsize=(5,5))
utils.show_group_stats_viz(d_train, 'gender')

In [ ]:
# Test partition
plt.figure(figsize=(5,5))
utils.show_group_stats_viz(d_test, 'gender')

## 4-5. Convert Dataset to TF Dataset

In [ ]:
# convert dataset from Pandas dataframes to TF dataset
batch_size = 128
diabetes_train_ds = utils.df_to_dataset(d_train, PREDICTOR_FIELD, batch_size=batch_size)
diabetes_val_ds = utils.df_to_dataset(d_val, PREDICTOR_FIELD, batch_size=batch_size)
diabetes_test_ds = utils.df_to_dataset(d_test, PREDICTOR_FIELD, batch_size=batch_size)

In [ ]:
# utils.py
import tensorflow as tf
import pandas as pd
import numpy as np

def df_to_dataset(df, predictor, batch_size):
    df = df.copy()
    labels = df.pop(predictor)
    # Convert 'object' columns to numerical if possible
    for col in df.select_dtypes(include=['object']):
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            pass  # Handle columns that cannot be converted
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

These messages are notifications indicating that the vocabulary size for some features has been inferred based on the number of elements in the corresponding vocabulary file.

In [ ]:
# We use this sample of the dataset to show transformations later
diabetes_batch = next(iter(diabetes_train_ds))[0]
def demo(feature_column, example_batch):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch))

In [ ]:
# Example values - replace these with your actual data
d_train = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6], 'target': [7, 8, 9]})
d_val = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6], 'target': [7, 8, 9]}) # Added this line
d_test = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6], 'target': [7, 8, 9]}) # Added this line
PREDICTOR_FIELD = 'target'

# convert dataset from Pandas dataframes to TF dataset
batch_size = 128
diabetes_train_ds = utils.df_to_dataset(d_train, PREDICTOR_FIELD, batch_size=batch_size)
diabetes_val_ds = utils.df_to_dataset(d_val, PREDICTOR_FIELD, batch_size=batch_size)
diabetes_test_ds = utils.df_to_dataset(d_test, PREDICTOR_FIELD, batch_size=batch_size)

## 4-6. Create Categorical Features with TF Feature Columns

In [ ]:
# we adding the directory
#!mkdir diabetes_vocab
vocab_file_list = utils.build_vocab_files(d_train, student_categorical_col_list)

In [ ]:
# we adding the directory
#!mkdir diabetes_vocab
# Check the columns in d_train
print(d_train.columns)

# Update student_categorical_col_list with valid column names from d_train
student_categorical_col_list = ['feature1', 'feature2']

vocab_file_list = utils.build_vocab_files(d_train, student_categorical_col_list)

In [ ]:
# view the list of vocab files
vocab_file_list

In [ ]:
from student_utils import create_tf_categorical_feature_cols

# Replace 'student_utils' with 'utils' assuming that's the correct import
tf_cat_col_list = create_tf_categorical_feature_cols(student_categorical_col_list)


## 4-7. Create Numerical Features

In [ ]:
for col in student_numerical_col_list:
    d_train[col] = d_train[col].astype('float32')
    d_val[col] = d_val[col].astype('float32')
    d_test[col] = d_test[col].astype('float32')

In [ ]:
for col in student_numerical_col_list:
    # Check if the column exists before accessing it
    if col in d_train.columns:
        d_train[col] = d_train[col].astype('float32')
    if col in d_val.columns:
        d_val[col] = d_val[col].astype('float32')
    if col in d_test.columns:
        d_test[col] = d_test[col].astype('float32')

In [ ]:
import student_utils
import functools
from student_utils import normalize_numeric_with_zscore, create_tf_numeric_feature

In [ ]:
def calculate_stats_from_train_data(df, col):
    mean = df[col].describe()['mean']
    std = df[col].describe()['std']
    return mean, std

def create_tf_numerical_feature_cols(numerical_col_list, train_df):
    tf_numeric_col_list = []
    for c in numerical_col_list:
        mean, std = calculate_stats_from_train_data(train_df, c)
        tf_numeric_feature = create_tf_numeric_feature(c, mean, std)
        tf_numeric_col_list.append(tf_numeric_feature)
    return tf_numeric_col_list

In [ ]:
tf_cont_col_list = create_tf_numerical_feature_cols(student_numerical_col_list, d_train)

## 4-8. Combine Features with DenseFeatures

In [ ]:
# concatenate categorical and numeric feature columns
claim_feature_columns = tf_cat_col_list + tf_cont_col_list

# combine with TF DenseFeatures
claim_feature_layer = tf.keras.layers.DenseFeatures(claim_feature_columns)

In [ ]:
def calculate_stats_from_train_data(df, col):
    # Check if column exists in DataFrame
    if col in df.columns:
        mean = df[col].describe()['mean']
        std = df[col].describe()['std']
        return mean, std
    else:
        # Handle missing column (e.g., return default values or raise an exception)
        return 0, 1  # Example: return 0 for mean and 1 for std

def create_tf_numerical_feature_cols(numerical_col_list, train_df):
    tf_numeric_col_list = []
    for c in numerical_col_list:
        # Calculate stats only if the column exists
        mean, std = calculate_stats_from_train_data(train_df, c)
        tf_numeric_feature = create_tf_numeric_feature(c, mean, std)
        tf_numeric_col_list.append(tf_numeric_feature)
    return tf_numeric_col_list

# 5. Build and Train the Model

The model is built with DenseFeatures and TF Probability Layers. The optimizer used is rmsprop. We train the model for 10 epochs.

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, MultiHeadAttention, Dropout

# Define Transformer Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Create Transformer-based model for EHR data
def build_transformer_model(input_shape, num_heads=4, embed_dim=32, ff_dim=128):
    inputs = tf.keras.Input(shape=input_shape)
    transformer_block = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)
    x = transformer_block(inputs)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)  # For binary classification (diabetes)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["AUC", "F1", "Precision", "Recall", "Accuracy"])
    return model


In [ ]:
!pip install graphviz
import graphviz

def visualize_transformer_model():
    # Create a new Digraph object for the structure
    dot = graphviz.Digraph(comment='Transformer-based Model')

    # Add input layer
    dot.node('Input', 'Input Layer\nShape: (input_shape)', shape='ellipse')

    # Transformer Block
    dot.node('Transformer', 'Transformer Block\n(Multi-Head Attention, FFN)', shape='box')
    dot.node('LayerNorm1', 'LayerNorm1', shape='box')
    dot.node('Dropout1', 'Dropout1', shape='box')
    dot.node('FFN', 'Feed Forward Network\n(Dense layers)', shape='box')
    dot.node('LayerNorm2', 'LayerNorm2', shape='box')
    dot.node('Dropout2', 'Dropout2', shape='box')

    # Dense and Dropout Layers after Transformer Block
    dot.node('Dense64', 'Dense Layer (64 units)\nActivation: ReLU', shape='box')
    dot.node('Dropout3', 'Dropout (0.5)', shape='box')

    # Output Layer
    dot.node('Output', 'Output Layer\nActivation: Sigmoid\n(Binary Classification)', shape='ellipse')

    # Connect the layers
    dot.edge('Input', 'Transformer')
    dot.edge('Transformer', 'LayerNorm1')
    dot.edge('LayerNorm1', 'Dropout1')
    dot.edge('Dropout1', 'FFN')
    dot.edge('FFN', 'LayerNorm2')
    dot.edge('LayerNorm2', 'Dropout2')
    dot.edge('Dropout2', 'Dense64')
    dot.edge('Dense64', 'Dropout3')
    dot.edge('Dropout3', 'Output')

    # Render the graph to a file or display it
    dot.render('transformer_model_structure', view=True, format='png')  # Save and open as a PNG file

# Call the function to generate the structure diagram
visualize_transformer_model()


In [ ]:
# Make sure graphviz is installed using: !pip install graphviz
import graphviz

def visualize_hybrid_cnn_transformer_model():
    # Create a new Digraph object for the structure
    dot = graphviz.Digraph(comment='Hybrid CNN-Transformer Model')

    # Add input layer
    dot.node('Input', 'Input Layer\nShape: (input_shape)', shape='ellipse')

    # CNN Block
    dot.node('Conv1', 'Conv2D Layer\n(32 filters, 3x3 kernel)\nActivation: ReLU', shape='box')
    dot.node('MaxPool1', 'MaxPooling2D\n(2x2 pool size)', shape='box')
    dot.node('Conv2', 'Conv2D Layer\n(64 filters, 3x3 kernel)\nActivation: ReLU', shape='box')
    dot.node('MaxPool2', 'MaxPooling2D\n(2x2 pool size)', shape='box')
    dot.node('Flatten', 'Flatten Layer', shape='box')

    # Transformer Block
    dot.node('Transformer', 'Transformer Block\n(Multi-Head Attention, FFN)', shape='box')
    dot.node('LayerNorm1', 'LayerNorm1', shape='box')
    dot.node('Dropout1', 'Dropout1', shape='box')
    dot.node('FFN', 'Feed Forward Network\n(Dense layers)', shape='box')
    dot.node('LayerNorm2', 'LayerNorm2', shape='box')
    dot.node('Dropout2', 'Dropout2', shape='box')

    # Dense and Dropout Layers after Transformer Block
    dot.node('Dense64', 'Dense Layer (64 units)\nActivation: ReLU', shape='box')
    dot.node('Dropout3', 'Dropout (0.5)', shape='box')

    # Output Layer
    dot.node('Output', 'Output Layer\nActivation: Sigmoid\n(Binary Classification)', shape='ellipse')

    # Connect the CNN layers
    dot.edge('Input', 'Conv1')
    dot.edge('Conv1', 'MaxPool1')
    dot.edge('MaxPool1', 'Conv2')
    dot.edge('Conv2', 'MaxPool2')
    dot.edge('MaxPool2', 'Flatten')

    # Connect CNN block to Transformer Block
    dot.edge('Flatten', 'Transformer')

    # Connect the layers within the Transformer block
    dot.edge('Transformer', 'LayerNorm1')
    dot.edge('LayerNorm1', 'Dropout1')
    dot.edge('Dropout1', 'FFN')
    dot.edge('FFN', 'LayerNorm2')
    dot.edge('LayerNorm2', 'Dropout2')

    # Connect Transformer block to final dense layers
    dot.edge('Dropout2', 'Dense64')
    dot.edge('Dense64', 'Dropout3')
    dot.edge('Dropout3', 'Output')

    # Render the graph to a file or display it
    dot.render('hybrid_cnn_transformer_model_structure', view=True, format='png')  # Save and open as a PNG file

# Call the function to generate the structure diagram
visualize_hybrid_cnn_transformer_model()


In [ ]:
def build_hybrid_cnn_transformer_model(input_shape):
    # Initialize model with CNN and Transformer layers
    model = initialize_model(input_shape)

    # Loop over the number of epochs
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Loop over each training example (x, y)
        for x, y in training_data:
            # Forward propagation
            # Pass inputs through the CNN block to extract local features
            cnn_features = cnn_block(x)
            print("Extracted local features from CNN")

            # Pass local features through the Transformer block
            transformer_output = transformer_block(cnn_features)
            print("Passed features through Transformer block")

            # Apply Dense layers and activation functions
            outputs = dense_layers(transformer_output)
            print("Applied Dense layers and activation functions")

            # Compute the loss
            loss = compute_loss(outputs, y)
            print(f"Loss computed: {loss}")

            # Backpropagation
            # Compute gradients of loss with respect to model parameters
            model = backpropagate(loss)
            print("Backpropagation completed, gradients updated")

    # Return the trained model
    return model


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers

# Define the function to create the DenseVariational layer
def get_dense_variational():
    # Define the prior distribution for the kernel and bias
    def prior(kernel_size, bias_size=0, dtype=None):
        n = kernel_size + bias_size
        return tf.keras.Sequential([
            tfp.layers.VariableLayer(2 * n, dtype=dtype),
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.Independent(
                    tfp.distributions.Normal(
                        loc=t[..., :n], scale=1e-5 + tf.nn.softplus(0.01 * t[..., n:])
                    ),
                    reinterpreted_batch_ndims=1,
                )
            ),
        ])

    # Define the posterior distribution for the kernel and bias
    def posterior(kernel_size, bias_size=0, dtype=None):
        n = kernel_size + bias_size
        return tf.keras.Sequential([
            tfp.layers.VariableLayer(2 * n, dtype=dtype),
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.Independent(
                    tfp.distributions.Normal(
                        loc=t[..., :n], scale=1e-5 + tf.nn.softplus(0.01 * t[..., n:])
                    ),
                    reinterpreted_batch_ndims=1,
                )
            ),
        ])

    # Create the DenseVariational layer
    return tfp.layers.DenseVariational(
        units=1, # Example output units
        make_prior_fn=None,
        make_posterior_fn=None,
        kl_weight=None,
        activation=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        kernel_prior_fn=None,
        bias_prior_fn=None,
        kernel_posterior_fn=None,
        bias_posterior_fn=None,
        kl_divergence_function=None
    )

In [ ]:
#Parallel
# Record the start time of the training
import time
start_time = time.time()

# Build and train the model
diabetes_model, history = build_diabetes_model(diabetes_train_ds, diabetes_val_ds, claim_feature_layer, epochs=20)

# Record the end time of the training
end_time = time.time()

# Calculate the training time in seconds
training_time_seconds = end_time - start_time

# Calculate the training time in minutes and seconds
training_time_minutes = training_time_seconds / 60

# Print the training time
print("Training time: {:.2f} seconds".format(training_time_seconds))
print("Training time: {:.2f} minutes".format(training_time_minutes))


fluctuations in validation MSE during training can be due to changes in dataset composition, data irregularities, model parameter tuning, random sampling effects, and environmental or noise effects. These factors interact to influence model performance across epochs.

# 6. Results

## 6-1. Uncertainty of Model Prediction

We use TF Probability Layer to build the model. As a result, we can extract the mean and standard deviation for each prediction the model made.

In [ ]:
# list of columns used as features
feature_list = student_categorical_col_list + student_numerical_col_list
#
diabetes_x_tst = dict(d_test[feature_list])
#
diabetes_yhat = diabetes_model(diabetes_x_tst)

# predict on the test dataset
preds = diabetes_model.predict(diabetes_test_ds)

In [ ]:
from student_utils import get_mean_std_from_preds
m, s = get_mean_std_from_preds(diabetes_yhat)

In [ ]:
# create a dict for building the dataframe
prob_outputs = {"pred": preds.flatten(),
                "actual_value": d_test['time_in_hospital'].values,
                "pred_mean": m.numpy().flatten(),
                "pred_std": s.numpy().flatten()}

# convert prob_outputs into a dataframe
prob_output_df = pd.DataFrame(prob_outputs)

In [ ]:
# check the results
# Start time calculation
start_time = time.time()

# Assuming selected_features_df is defined in your code
# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(tabulate(selected_features_df.head(), headers='keys', tablefmt='pretty', showindex=False))

# Calculate and print the number of rows and columns
num_rows, num_columns = selected_features_df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


## 6-2. Patient Selection

We use the mean of the prediction to perform binary classification on each patients.

In [ ]:
from student_utils import get_student_binary_prediction

student_binary_prediction = get_student_binary_prediction(prob_output_df, 'pred_mean')

In [ ]:
def add_pred_to_test(test_df, pred_np, demo_col_list):
    """
    Create a dataframe for the test dataset with 'score' and 'label_value'.
    'score' is the binary prediction that a patient would spend >= 5 days in the hospital
    'label_value' is the ground truth that a patient spent >= 5 days in the hospital
    Args:
        test_df - the test dataset
        pred_np - the binary prediction of the test dataset by the model
        demo_col_list: the categorical feature names
    Output:
        test_df - dataframe of the test dataset (with score and label_value)
    """

    # Create a copy of the dataframe to avoid SettingWithCopyWarning
    test_df = test_df.copy()

    # convert all values of the categorical columns into strings
    for c in demo_col_list:
        test_df[c] = test_df[c].astype(str)

    # model binary prediction
    test_df['score'] = pred_np

    # actual labels
    test_df['label_value'] = test_df['time_in_hospital'].apply(lambda x: 1 if x >= 5 else 0)

    return test_df

# Use the modified function
pred_test_df = add_pred_to_test(d_test, student_binary_prediction, ['race', 'gender'])


In [ ]:
# Start time calculation
start_time = time.time()

# Assuming pred_test_df is defined in your code
# Print the selected columns of the first few rows of the DataFrame
print("Selected columns of the first few rows of the DataFrame:")
print(tabulate(pred_test_df[['patient_nbr', 'gender', 'race', 'time_in_hospital', 'score', 'label_value']].head(), headers='keys', tablefmt='pretty', showindex=False))

# End time calculation
end_time = time.time()

# Calculate the duration
duration_seconds = end_time - start_time

# Print the duration in seconds
print("Duration in seconds:", duration_seconds)


# 7. Model Evaluation

## 7-1. Precision and Recall

Precision is the ratio of the number of true positives to the sum of the true positives and false positives. Recall is the ratio of the number of true positives to the sum of the true positives and false negatives. There's a tradeoff between them. If we increase the precision (we say no to the data that we're less confident in classifying as positive), we will classfy more true positives as negative and as a result decreasing the recall. On the other hand, if we increase the recall (by saying yes to the data that we're less confident in classifying it as as negative), we will calssify more true negative data as positive and as a result decreasing the precision.

To further improve the model's performance, we could focus on enhancing the recall without significantly sacrificing precision. One approach could be to adjust the classification threshold to favor sensitivity over specificity, thereby increasing the likelihood of capturing more positive cases. Additionally, we might explore feature engineering techniques to extract more relevant information from the data, which could potentially improve the model's ability to identify positive cases. Moreover, experimenting with different algorithms or model architectures might yield better results. Finally, incorporating techniques such as ensemble learning or model stacking could also help in boosting performance by combining the strengths of multiple models.

In [ ]:
from tabulate import tabulate
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

y_true = pred_test_df['label_value']
y_pred = pred_test_df['score']

# AUC, F1, precision and recall
# Summary

auc = roc_auc_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Metric': ['AUC', 'F1 score', 'Precision', 'Recall'],
    'Value': [auc, f1, precision, recall]
})

# Convert DataFrame to a list of lists for tabulate
results_list = results_df.values.tolist()

# Print the results using tabulate
print(tabulate(results_list, headers=['Metric ', 'Value'], tablefmt='pretty'))


In [ ]:
plt.figure(figsize=(5,15))
import matplotlib.pyplot as plt

# Extract loss history from the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Extract performance metrics from the results DataFrame
metrics = results_df['Metric']
values = results_df['Value']

# Plotting
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# Plot training and validation loss
axes[0].plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss', color='blue')
axes[0].plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss', color='orange')
axes[0].set_title('Training and Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()

# Plot evaluation metrics
axes[1].bar(metrics, values, color=['blue', 'blue', 'blue', 'blue'])
axes[1].set_title('Evaluation Metrics')
axes[1].set_xlabel('Metric')
axes[1].set_ylabel('Value')
axes[1].set_ylim(0, 1)  # Setting y-axis limit to better visualize AUC
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd

# Assuming y_true and y_pred are defined elsewhere in your code
# Calculate the Kappa score between the predictions and the actual data
kappa = cohen_kappa_score(y_true, y_pred)

# Define the levels of kappa
kappa_levels = {
    'Excellent': (0.81, 1.0),
    'Very Good': (0.61, 0.8),
    'Good': (0.41, 0.6),
    'Fair': (0.21, 0.409),
    'Poor': (0.0, 0.2)
}

# Determine the level of the kappa score
level = None
for category, (lower, upper) in kappa_levels.items():
    if lower <= kappa <= upper:
        level = category
        break

# Create a DataFrame to display the kappa score and its level
results = pd.DataFrame({
    "Metric": ["Kappa Score"],
    "Value": [kappa],
    "Level": [level]
})

# Print the results using tabulate
from tabulate import tabulate
print(tabulate(results, headers="keys", tablefmt="pretty"))


 The levels of results that can be obtained, they are as follows:

    Excellent: If the Kappa score value is between 0.81 and 1.0.
    Very Good: If the Kappa score value is between 0.61 and 0.8.
    Good: If the Kappa score value is between 0.41 and 0.6.
    Fair: If the Kappa score value is between 0.21 and 0.4.
    Poor: If the Kappa score value is between 0.0 and 0.2.

These levels assess the degree of agreement between predictions and actual data, where higher Kappa values indicate better agreement between data and predictions.


## 7-2. Biases



In [ ]:
# Aequitas
from aequitas.preprocessing import preprocess_input_df
from aequitas.group import Group
from aequitas.plotting import Plot
from aequitas.bias import Bias
from aequitas.fairness import Fairness


ae_subset_df = pred_test_df[['race', 'gender', 'score', 'label_value']]
ae_df, _ = preprocess_input_df(ae_subset_df)

g = Group()
xtab, _ = g.get_crosstabs(ae_df)
absolute_metrics = g.list_absolute_metrics(xtab)
clean_xtab = xtab.fillna(-1)

aqp = Plot()
b = Bias()

In [ ]:
# test reference group with Caucasian Male
bdf = b.get_disparity_predefined_groups(clean_xtab,
                                        original_df=ae_df,
                                        ref_groups_dict={'race':'Caucasian', 'gender':'Male'},
                                        alpha=0.05,
                                        check_significance=False)

f = Fairness()
fdf = f.get_group_value_fairness(bdf)

In [ ]:
# fairness: false positive rate
fpr_fairness = aqp.plot_fairness_group(fdf, group_metric='fpr', title=True)

In [ ]:
import matplotlib.pyplot as plt

fpr_fairness = aqp.plot_fairness_group(fdf, group_metric='fpr', title=True)

#
plt.xticks(rotation=90)

plt.show()





In [ ]:
# fairness: false negative rate
fnr_fairness = aqp.plot_fairness_group(fdf, group_metric='fnr', title=True)

## 7-3. Disparity

From the figures below we see that compared to Caucasian (the reference group), all other races have a higher false positive rate.

On the other hand, we don't see this in gender. Gender doesn't not have much influence on the false positive rate.

In [ ]:
# FPR disparity of race
fpr_disparity = aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='race')

If the false positive rate (FPR) is lower, it means the model is better at classifying negative samples correctly as negatives, indicating better performance in predicting negative outcomes. Therefore, if the value is lower (such as 0.87), it signifies better performance compared to a higher value (like 1.96), hence the value 0.87 is better.

In [ ]:
# FPR disparity of gender
fpr_disparity = aqp.plot_disparity(bdf, group_metric='fpr_disparity', attribute_name='gender')